## Mount drive and import packages

In [1]:
from typing import List, Tuple, Dict, Optional, Set
from enum import Enum
import os
import pickle
import gc
import pandas as pd
from Bio import SeqIO
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import esm
from icecream import ic
import random
from collections import Counter
import requests
from urllib.parse import quote
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import sys

sys.path.append('/workspace/protein_lm/tokenizer')
from tokenizer import EsmTokenizer, PTMTokenizer

sys.path.append('/workspace/protein_lm/modeling/scripts')
from infer import PTMMamba

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Define token alphabet

In [2]:
tokens = [
            "<cls>",
            "<pad>",
            "<eos>",
            "<unk>",
            ".",
            "-",
            "<null_1>",
            "<mask>",
            "L",
            "A",
            "G",
            "V",
            "S",
            "E",
            "R",
            "T",
            "I",
            "D",
            "P",
            "K",
            "Q",
            "N",
            "F",
            "Y",
            "M",
            "H",
            "W",
            "C",
            "X",
            "B",
            "U",
            "Z",
            "O",
            "PTM",
            "<N-linked (GlcNAc...) asparagine>",
            "<Pyrrolidone carboxylic acid>",
            "<Phosphoserine>",
            "<Phosphothreonine>",
            "<N-acetylalanine>",
            "<N-acetylmethionine>",
            "<N6-acetyllysine>",
            "<Phosphotyrosine>",
            "<S-diacylglycerol cysteine>",
            "<N6-(pyridoxal phosphate)lysine>",
            "<N-acetylserine>",
            "<N6-carboxylysine>",
            "<N6-succinyllysine>",
            "<S-palmitoyl cysteine>",
            "<O-(pantetheine 4'-phosphoryl)serine>",
            "<Sulfotyrosine>",
            "<O-linked (GalNAc...) threonine>",
            "<Omega-N-methylarginine>",
            "<N-myristoyl glycine>",
            "<4-hydroxyproline>",
            "<Asymmetric dimethylarginine>",
            "<N5-methylglutamine>",
            "<4-aspartylphosphate>",
            "<S-geranylgeranyl cysteine>",
            "<4-carboxyglutamate>",
        ]
token_to_index = {token: i for i, token in enumerate(tokens)}

## Get uniprot ids of proteins that are positive for druggability

In [3]:
positive_druggability_df_path = '/workspace/protein_lm/evaluation/nonptm_vs_ptm_classification/data/scraped_drugptm_table_with_uniprot_ids.csv'
positive_druggability_df = pd.read_csv(positive_druggability_df_path)
positive_druggability_ids = set(positive_druggability_df['UniProtID'])

## Get uniprot ids of proteins that are positive for disease

In [4]:
positive_disease_df_path = '/workspace/protein_lm/evaluation/nonptm_vs_ptm_classification/data/scraped_snpptm_table_with_uniprot_ids.csv'
positive_disease_df = pd.read_csv(positive_disease_df_path)
positive_disease_ids = set(positive_disease_df['UniProtID'])

## Get uniprot ids of proteins that are negative for druggability

In [5]:
negative_druggability_df_path = '/workspace/protein_lm/evaluation/nonptm_vs_ptm_classification/data/non_druggable_uniprot_ids.csv'
negative_druggability_df = pd.read_csv(negative_druggability_df_path)
negative_druggability_ids = set(negative_druggability_df['uniprot_id'])

## Get uniprot ids of proteins that are negative for disease

In [6]:
negative_disease_txt_path = '/workspace/protein_lm/evaluation/nonptm_vs_ptm_classification/data/non_disease_uniprot_ids.txt'
with open(negative_disease_txt_path, 'r') as file:
  negative_disease_ids = set([line.strip() for line in file])

In [7]:
all_protein_ids = positive_druggability_ids | positive_disease_ids | negative_druggability_ids | negative_disease_ids

## Download all protein sequences

In [8]:
def download_protein_sequence(uniprot_id, file_path):
    base_url = "https://rest.uniprot.org/uniprotkb/"
    headers = {"Accept": "text/plain"}
    # Construct the query for a single UniProt ID
    url = f"{base_url}/{uniprot_id}.fasta"
    response = requests.get(url, headers=headers)
    if response.status_code == 200 and response.text:
        with open(file_path, 'a') as file:
            file.write(response.text)
            # print(f"Downloaded sequence for UniProt ID: {uniprot_id}")
    else:
        print(f"Failed to download sequence for {uniprot_id}. Status code: {response.status_code}, Response: {response.text}")

def download_protein_sequences(uniprot_ids, file_path):
    for uniprot_id in tqdm(uniprot_ids):
        download_protein_sequence(uniprot_id, file_path)

In [9]:
all_ids = list(positive_druggability_ids | positive_disease_ids | negative_druggability_ids | negative_disease_ids)
all_sequences_file_path = '/workspace/protein_lm/evaluation/nonptm_vs_ptm_classification/data/all_sequences.fasta'
if os.path.exists(all_sequences_file_path):
	os.remove(all_sequences_file_path)
ic(len(all_ids))

ic| len(all_ids): 2466


2466

In [ ]:
download_protein_sequences(all_ids, all_sequences_file_path)

In [ ]:
def fasta_to_dataframe(fasta_file):
    ids = []
    sequences = []
    for record in SeqIO.parse(fasta_file, "fasta"):
        uniprot_id = record.id.split('|')[1]
        sequence = str(record.seq)
        ids.append(uniprot_id)
        sequences.append(sequence)
    df = pd.DataFrame({
        'UniProt ID': ids,
        'Sequence': sequences
    })
    return df

# wt_sequences_df = fasta_to_dataframe(all_sequences_file_path)
# ic(len(wt_sequences_df))
# wt_sequences_df.head()

## Augment map uniprot ids to ptm_data.csv rows using their sequences

In [11]:
ptm_wt_df = pd.read_csv('/workspace/protein_lm/evaluation/nonptm_vs_ptm_classification/data/ptm_data.csv')
ptm_wt_df = ptm_wt_df[['AC_ID', 'wt_seq', 'ptm_seq']]
ptm_wt_df.head()

,AC_ID,wt_seq,ptm_seq
0,O60296,MSQSQNAIFTSPTGEENLMNSNHRDSESITDVCSNEDLPEVELVSL...,MSQSQNAIFTSPTGEENLMNSNHRDSESITDVCSNEDLPEVELVSL...
1,B5EXH1,MTSRNYLLLTPGPLTTSRTVKEAMLFDSCTWDDDYNIGVVEQIRQQ...,MTSRNYLLLTPGPLTTSRTVKEAMLFDSCTWDDDYNIGVVEQIRQQ...
2,Q9BXI9,MQWLRVRESPGEATGHRVTMGTAALGPVWAALLLFLLMCEIPMVEL...,MQWLRVRESPGEATGHRVTMGTAALGPVWAALLLFLLMCEIPMVEL...
3,Q6ZFW0,MAASKGNAAAAACALVLVLLAVGAEAQGGGGGECVPQLNRLLACRA...,MAASKGNAAAAACALVLVLLAVGAEAQGGGGGECVPQLNRLLACRA...
4,Q3SYP2,MLGITVLAAILACASSCGDPTFPPNLSARVVGGEDAVPNSWPWQVS...,MLGITVLAAILACASSCGDPTFPP<N-linked (GlcNAc...) ...


In [ ]:
ptm_wt_df_seqs = set(ptm_wt_df['wt_seq'])
wt_sequences_df_seqs = set(wt_sequences_df['Sequence'])

ic(len(ptm_wt_df_seqs))
ic(len(wt_sequences_df_seqs))
ic(len(ptm_wt_df_seqs.intersection(wt_sequences_df_seqs)))

ic| len(ptm_wt_df_seqs): 79707
ic| len(wt_sequences_df_seqs): 2461
ic| len(ptm_wt_df_seqs.intersection(wt_sequences_df_seqs)): 1682


1682

In [ ]:
ac_ids = set(ptm_wt_df['AC_ID'])
ic(len(ac_ids))
ic(len(all_protein_ids))
ic(len(ac_ids.intersection(all_protein_ids)))

ic| len(ac_ids): 79707
ic| len(all_protein_ids): 2466
ic| len(ac_ids.intersection(all_protein_ids)): 1619


1619

In [ ]:
seqs_df = df_joined = pd.merge(wt_sequences_df, ptm_wt_df, left_on='Sequence', right_on='wt_seq')

seqs_df.head()

NameError: name 'wt_sequences_df' is not defined

In [ ]:
seqs_df.rename(columns={'UniProt ID': 'uniprot_id', "AC_ID": "accession_id"}, inplace=True)
seqs_df.drop('Sequence', axis=1, inplace=True)

embeddings_dir = '/workspace/protein_lm/evaluation/nonptm_vs_ptm_classification/data/embeddings'
seqs_df['one_hot_embedding_path'] = seqs_df['uniprot_id'].apply(lambda x: f"{embeddings_dir}/{x}_one_hot.pt")
seqs_df['one_hot_ptm_embedding_path'] = seqs_df['uniprot_id'].apply(lambda x: f"{embeddings_dir}/{x}_one_hot_ptm.pt")
seqs_df['esm_650m_embedding_path'] = seqs_df['uniprot_id'].apply(lambda x: f"{embeddings_dir}/{x}_esm_650m.pt")
seqs_df['esm_3b_embedding_path'] = seqs_df['uniprot_id'].apply(lambda x: f"{embeddings_dir}/{x}_esm_3b.pt")
seqs_df['ptm_mamba_embedding_path'] = seqs_df['uniprot_id'].apply(lambda x: f"{embeddings_dir}/{x}_ptm_mamba.pt")

seqs_df.head()

,uniprot_id,accession_id,wt_seq,ptm_seq,one_hot_embedding_path,one_hot_ptm_embedding_path,esm_650m_embedding_path,esm_3b_embedding_path,ptm_mamba_embedding_path,is_druggable,is_disease,is_part_of_druggability_dataset,is_part_of_disease_dataset
0,P07107,P07107,MSQAEFDKAAEEVKHLKTKPADEEMLFIYSHYKQATVGDINTERPG...,M<N-acetylserine>QAEFDKAAEEVKHL<N6-succinyllys...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,True,False,True,False
1,O00590,O00590,MAATASPQPLATEDADSENSSFYYYDYLDEVAFMLCRKDAVVSFGK...,MAATASPQPLATEDADSE<N-linked (GlcNAc...) aspara...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,False,True,False,True
2,O75716,O75716,MGHALCVCSRGTVIIDNKRYLFIQKLGEGGFSYVDLVEGLHDGHFY...,MGHALCVCSRGTVIIDNKRYLFIQKLGEGGFSYVDLVEGLHDGHFY...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,True,False,True,False
3,P23560,Q5IS78,MTILFLTMVISYFGCMKAAPMKEANIRGQGGLAYPGVRTHGTLESV...,MTILFLTMVISYFGCMKAAPMKEANIRGQGGLAYPGVRTHGTLESV...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,False,True,False,True
4,Q99471,Q99471,MAQSINITELNLPQLEMLKNQLDQEVEFLSTSIAQLKVVQTKYVEA...,M<N-acetylalanine>QSINITELNLPQLEMLKNQLDQEVEFLS...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,False,False,True,False


In [ ]:
seqs_df['is_druggable'] = seqs_df['uniprot_id'].apply(lambda x: x in positive_druggability_ids)
seqs_df['is_disease'] = seqs_df['uniprot_id'].apply(lambda x: x in positive_disease_ids)

seqs_df['is_part_of_druggability_dataset'] = seqs_df['uniprot_id'].apply(lambda x: x in positive_druggability_ids or x in negative_druggability_ids)
seqs_df['is_part_of_disease_dataset'] = seqs_df['uniprot_id'].apply(lambda x: x in positive_disease_ids or x in negative_disease_ids)

seqs_df.head()

,uniprot_id,accession_id,wt_seq,ptm_seq,one_hot_embedding_path,one_hot_ptm_embedding_path,esm_650m_embedding_path,esm_3b_embedding_path,ptm_mamba_embedding_path,is_druggable,is_disease,is_part_of_druggability_dataset,is_part_of_disease_dataset
0,P07107,P07107,MSQAEFDKAAEEVKHLKTKPADEEMLFIYSHYKQATVGDINTERPG...,M<N-acetylserine>QAEFDKAAEEVKHL<N6-succinyllys...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,True,False,True,False
1,O00590,O00590,MAATASPQPLATEDADSENSSFYYYDYLDEVAFMLCRKDAVVSFGK...,MAATASPQPLATEDADSE<N-linked (GlcNAc...) aspara...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,False,True,False,True
2,O75716,O75716,MGHALCVCSRGTVIIDNKRYLFIQKLGEGGFSYVDLVEGLHDGHFY...,MGHALCVCSRGTVIIDNKRYLFIQKLGEGGFSYVDLVEGLHDGHFY...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,True,False,True,False
3,P23560,Q5IS78,MTILFLTMVISYFGCMKAAPMKEANIRGQGGLAYPGVRTHGTLESV...,MTILFLTMVISYFGCMKAAPMKEANIRGQGGLAYPGVRTHGTLESV...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,False,True,False,True
4,Q99471,Q99471,MAQSINITELNLPQLEMLKNQLDQEVEFLSTSIAQLKVVQTKYVEA...,M<N-acetylalanine>QSINITELNLPQLEMLKNQLDQEVEFLS...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,False,False,True,False


In [ ]:
uniprot_id_to_index = {uniprot_id: i for i, uniprot_id in enumerate(seqs_df['uniprot_id'])}

In [ ]:
seqs_df.to_csv('/workspace/protein_lm/evaluation/nonptm_vs_ptm_classification/data/seqs_df.csv', index=False)

In [12]:
seqs_df = pd.read_csv('/workspace/protein_lm/evaluation/nonptm_vs_ptm_classification/data/seqs_df.csv')

seqs_df.head()

,uniprot_id,accession_id,wt_seq,ptm_seq,one_hot_embedding_path,one_hot_ptm_embedding_path,esm_650m_embedding_path,esm_3b_embedding_path,ptm_mamba_embedding_path,is_druggable,is_disease,is_part_of_druggability_dataset,is_part_of_disease_dataset,is_train
0,P07107,P07107,MSQAEFDKAAEEVKHLKTKPADEEMLFIYSHYKQATVGDINTERPG...,M<N-acetylserine>QAEFDKAAEEVKHL<N6-succinyllys...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,True,False,True,False,True
1,O00590,O00590,MAATASPQPLATEDADSENSSFYYYDYLDEVAFMLCRKDAVVSFGK...,MAATASPQPLATEDADSE<N-linked (GlcNAc...) aspara...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,False,True,False,True,True
2,O75716,O75716,MGHALCVCSRGTVIIDNKRYLFIQKLGEGGFSYVDLVEGLHDGHFY...,MGHALCVCSRGTVIIDNKRYLFIQKLGEGGFSYVDLVEGLHDGHFY...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,True,False,True,False,True
3,P23560,Q5IS78,MTILFLTMVISYFGCMKAAPMKEANIRGQGGLAYPGVRTHGTLESV...,MTILFLTMVISYFGCMKAAPMKEANIRGQGGLAYPGVRTHGTLESV...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,False,True,False,True,True
4,Q99471,Q99471,MAQSINITELNLPQLEMLKNQLDQEVEFLSTSIAQLKVVQTKYVEA...,M<N-acetylalanine>QSINITELNLPQLEMLKNQLDQEVEFLS...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,False,False,True,False,True


## Divide datasets into train and test sets using mmmseqs2

In [ ]:
drugptm_seqs_df = seqs_df[seqs_df['is_part_of_druggability_dataset']]
snpptm_seqs_df = seqs_df[seqs_df['is_part_of_disease_dataset']]

In [ ]:
# iteeate over the rows of drugptm_seqs_df and copy all wt_seqs to a fasta file
drugptm_seqs_file_path = '/workspace/protein_lm/evaluation/nonptm_vs_ptm_classification/data/clustering/drugptm_sequences.fasta'

with open(drugptm_seqs_file_path, 'a') as file:
	for index, row in drugptm_seqs_df.iterrows():
		file.write(f">{row['uniprot_id']}\n{row['wt_seq']}\n")


disease_seqs_file_path = '/workspace/protein_lm/evaluation/nonptm_vs_ptm_classification/data/clustering/disease_sequences.fasta'
with open(disease_seqs_file_path, 'a') as file:
	for index, row in snpptm_seqs_df.iterrows():
		file.write(f">{row['uniprot_id']}\n{row['wt_seq']}\n")

In [ ]:
%cd /workspace/protein_lm/evaluation/nonptm_vs_ptm_classification/data/clustering

/workspace/protein_lm/evaluation/nonptm_vs_ptm_classification/data/clustering


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
!mmseqs createdb /workspace/protein_lm/evaluation/nonptm_vs_ptm_classification/data/clustering/drugptm_sequences.fasta drugptmProteinDB

drugptmProteinDB exists and will be overwritten
createdb /workspace/protein_lm/evaluation/nonptm_vs_ptm_classification/data/clustering/drugptm_sequences.fasta drugptmProteinDB 

MMseqs Version:       	abab073141f0bba788aed068b376de99c4c3519a
Database type         	0
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[607] 0s 3ms
Time for merging to drugptmProteinDB_h: 0h 0m 0s 1ms
Time for merging to drugptmProteinDB: 0h 0m 0s 2ms
Database type: Aminoacid
Time for processing: 0h 0m 0s 10ms


In [ ]:
# !mmseqs cluster drugptmProteinDB drugptmClusters tmp --min-seq-id 0.3 -c 0.5 --cov-mode 1

!mmseqs cluster drugptmProteinDB drugptmClusters tmp --min-seq-id 0.3 -c 0.5 --cov-mode 1 --cluster-mode 0

cluster drugptmProteinDB drugptmClusters tmp --min-seq-id 0.3 -c 0.5 --cov-mode 1 --cluster-mode 0 

MMseqs Version:                     	abab073141f0bba788aed068b376de99c4c3519a
Substitution matrix                 	aa:blosum62.out,nucl:nucleotide.out
Seed substitution matrix            	aa:VTML80.out,nucl:nucleotide.out
Sensitivity                         	4
k-mer length                        	0
Target search mode                  	0
k-score                             	seq:2147483647,prof:2147483647
Alphabet size                       	aa:21,nucl:5
Max sequence length                 	65535
Max results per query               	20
Split database                      	0
Split mode                          	2
Split memory limit                  	0
Coverage threshold                  	0.5
Coverage mode                       	1
Compositional bias                  	1
Compositional bias                  	1
Diagonal scoring                    	true
Exact k-mer matching                	0
Mas

In [ ]:
!mmseqs createtsv drugptmProteinDB drugptmProteinDB drugptmClusters drugptmClusters.tsv

createtsv drugptmProteinDB drugptmProteinDB drugptmClusters drugptmClusters.tsv 

MMseqs Version:                 	abab073141f0bba788aed068b376de99c4c3519a
First sequence as representative	false
Target column                   	1
Add full header                 	false
Sequence source                 	0
Database output                 	false
Threads                         	256
Compressed                      	0
Verbosity                       	3

Time for merging to drugptmClusters.tsv: 0h 0m 0s 11ms
Time for processing: 0h 0m 0s 126ms


In [ ]:
def read_clusters_dict_from_tsv(tsv_path):
    clusters = {}
    with open(tsv_path) as f:
        for line in f:
            parts = line.strip().split("\t")
            cluster_id = parts[0]
            sequence_id = parts[1]
            if cluster_id not in clusters:
                clusters[cluster_id] = set()
            clusters[cluster_id].add(sequence_id)
    return clusters

disease_tsv_path = "/workspace/protein_lm/evaluation/nonptm_vs_ptm_classification/data/clustering/drugptmClusters.tsv"
drugptm_clusters = read_clusters_dict_from_tsv(disease_tsv_path)

In [ ]:
def random_split_clusters(clusters):
    random.seed(42)

    cluster_ids = list(clusters.keys())
    random.shuffle(cluster_ids)

    num_test = int(len(cluster_ids) * 0.2)
    test_clusters = cluster_ids[:num_test]
    train_clusters = cluster_ids[num_test:]

    test_ids = {seq_id for cluster_id in test_clusters for seq_id in clusters[cluster_id]}
    train_ids = {seq_id for cluster_id in train_clusters for seq_id in clusters[cluster_id]}

    return train_ids, test_ids

drugptm_train_protein_ids, drugptm_test_protein_ids = random_split_clusters(drugptm_clusters)

In [ ]:
!mmseqs createdb /workspace/protein_lm/evaluation/nonptm_vs_ptm_classification/data/clustering/disease_sequences.fasta diseaseProteinDB

diseaseProteinDB exists and will be overwritten
createdb /workspace/protein_lm/evaluation/nonptm_vs_ptm_classification/data/clustering/disease_sequences.fasta diseaseProteinDB 

MMseqs Version:       	abab073141f0bba788aed068b376de99c4c3519a
Database type         	0
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[1011] 0s 4ms
Time for merging to diseaseProteinDB_h: 0h 0m 0s 2ms
Time for merging to diseaseProteinDB: 0h 0m 0s 4ms
Database type: Aminoacid
Time for processing: 0h 0m 0s 16ms


In [ ]:
!mmseqs cluster diseaseProteinDB diseaseClusters tmp --min-seq-id 0.3 -c 0.5 --cov-mode 1 --cluster-mode 0

# !mmseqs cluster diseaseProteinDB diseaseClusters tmp --min-seq-id 0.3 -c 0.5 --cov-mode 1

cluster diseaseProteinDB diseaseClusters tmp --min-seq-id 0.3 -c 0.5 --cov-mode 1 --cluster-mode 0 

MMseqs Version:                     	abab073141f0bba788aed068b376de99c4c3519a
Substitution matrix                 	aa:blosum62.out,nucl:nucleotide.out
Seed substitution matrix            	aa:VTML80.out,nucl:nucleotide.out
Sensitivity                         	4
k-mer length                        	0
Target search mode                  	0
k-score                             	seq:2147483647,prof:2147483647
Alphabet size                       	aa:21,nucl:5
Max sequence length                 	65535
Max results per query               	20
Split database                      	0
Split mode                          	2
Split memory limit                  	0
Coverage threshold                  	0.5
Coverage mode                       	1
Compositional bias                  	1
Compositional bias                  	1
Diagonal scoring                    	true
Exact k-mer matching                	0
Mas

In [ ]:
!mmseqs createtsv diseaseProteinDB diseaseProteinDB diseaseClusters diseaseClusters.tsv

createtsv diseaseProteinDB diseaseProteinDB diseaseClusters diseaseClusters.tsv 

MMseqs Version:                 	abab073141f0bba788aed068b376de99c4c3519a
First sequence as representative	false
Target column                   	1
Add full header                 	false
Sequence source                 	0
Database output                 	false
Threads                         	256
Compressed                      	0
Verbosity                       	3

Time for merging to diseaseClusters.tsv: 0h 0m 0s 12ms
Time for processing: 0h 0m 0s 116ms


In [ ]:
disease_tsv_path = "/workspace/protein_lm/evaluation/nonptm_vs_ptm_classification/data/clustering/diseaseClusters.tsv"
disease_clusters = read_clusters_dict_from_tsv(disease_tsv_path)

In [ ]:
disease_train_protein_ids, disease_test_protein_ids = random_split_clusters(disease_clusters)

In [ ]:
ic(len(drugptm_train_protein_ids))
ic(len(drugptm_test_protein_ids))
ic(len(disease_train_protein_ids))
ic(len(disease_test_protein_ids))

ic(len(set(drugptm_train_protein_ids).intersection(drugptm_test_protein_ids)))
ic(len(set(disease_train_protein_ids).intersection(disease_test_protein_ids)))

ic(list(drugptm_train_protein_ids)[:5])
ic(list(drugptm_test_protein_ids)[:5])
ic(list(disease_train_protein_ids)[:5])
ic(list(disease_test_protein_ids)[:5])

ic| len(drugptm_train_protein_ids): 552
ic| len(drugptm_test_protein_ids): 144
ic| len(disease_train_protein_ids): 809
ic| len(disease_test_protein_ids): 221
ic| len(set(drugptm_train_protein_ids).intersection(drugptm_test_protein_ids)): 0
ic| len(set(disease_train_protein_ids).intersection(disease_test_protein_ids)): 0
ic| list(drugptm_train_protein_ids)[:5]: ['P21457', 'P11142', 'P00875', 'Q04760', 'P07814']
ic| list(drugptm_test_protein_ids)[:5]: ['P80386', 'P31939', 'Q9X2A5', 'Q8TD30', 'P35475']
ic| list(disease_train_protein_ids)[:5]: ['P13866', 'Q8N5U6', 'Q8IV20', 'Q7Z7F7', 'P78346']
ic| list(disease_test_protein_ids)[:5]: ['Q969V6', 'P62955', 'Q3MIW9', 'O95613', 'Q9NTG1']


['Q969V6', 'P62955', 'Q3MIW9', 'O95613', 'Q9NTG1']

In [ ]:
seqs_df['is_train'] = seqs_df['uniprot_id'].apply(lambda x: x in drugptm_train_protein_ids or x in disease_train_protein_ids)

seqs_df.head()

,uniprot_id,accession_id,wt_seq,ptm_seq,one_hot_embedding_path,one_hot_ptm_embedding_path,esm_650m_embedding_path,esm_3b_embedding_path,ptm_mamba_embedding_path,is_druggable,is_disease,is_part_of_druggability_dataset,is_part_of_disease_dataset,is_train
0,P07107,P07107,MSQAEFDKAAEEVKHLKTKPADEEMLFIYSHYKQATVGDINTERPG...,M<N-acetylserine>QAEFDKAAEEVKHL<N6-succinyllys...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,True,False,True,False,True
1,O00590,O00590,MAATASPQPLATEDADSENSSFYYYDYLDEVAFMLCRKDAVVSFGK...,MAATASPQPLATEDADSE<N-linked (GlcNAc...) aspara...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,False,True,False,True,True
2,O75716,O75716,MGHALCVCSRGTVIIDNKRYLFIQKLGEGGFSYVDLVEGLHDGHFY...,MGHALCVCSRGTVIIDNKRYLFIQKLGEGGFSYVDLVEGLHDGHFY...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,True,False,True,False,True
3,P23560,Q5IS78,MTILFLTMVISYFGCMKAAPMKEANIRGQGGLAYPGVRTHGTLESV...,MTILFLTMVISYFGCMKAAPMKEANIRGQGGLAYPGVRTHGTLESV...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,False,True,False,True,True
4,Q99471,Q99471,MAQSINITELNLPQLEMLKNQLDQEVEFLSTSIAQLKVVQTKYVEA...,M<N-acetylalanine>QSINITELNLPQLEMLKNQLDQEVEFLS...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,/workspace/protein_lm/evaluation/nonptm_vs_ptm...,False,False,True,False,True


In [ ]:
seqs_df.to_csv('/workspace/protein_lm/evaluation/nonptm_vs_ptm_classification/data/seqs_df.csv', index=False)

## Create one-hot embeddings

In [24]:
tokenizer = PTMTokenizer()

for i, row in tqdm(seqs_df.iterrows()):
    uniprot_id = row['uniprot_id']
    wt_seq = row['wt_seq']
    one_hot_embedding_path = row['one_hot_embedding_path']
    one_hot_vector = tokenizer(wt_seq, return_tensor=True)
    padded_one_hot_vector = F.pad(one_hot_vector, (0, 7000 - one_hot_vector.shape[0]), "constant", 0)
    torch.save(padded_one_hot_vector, one_hot_embedding_path)

1686it [00:00, 2647.85it/s]


## Create one-hot PTM embeddings

In [25]:
tokenizer = PTMTokenizer()

for i, row in tqdm(seqs_df.iterrows()):
    uniprot_id = row['uniprot_id']
    ptm_seq = row['ptm_seq']
    one_hot_ptm_embedding_path = row['one_hot_ptm_embedding_path']
    one_hot_ptm_vector = tokenizer(ptm_seq, return_tensor=True)
    padded_one_hot_ptm_vector = F.pad(one_hot_ptm_vector, (0, 7000 - one_hot_ptm_vector.shape[0]), "constant", 0)
    torch.save(padded_one_hot_ptm_vector, one_hot_ptm_embedding_path)

1686it [00:00, 2615.06it/s]


## Create ESM 650M Embeddings

In [ ]:
esm_650m_model, esm_650m_alphabet = esm.pretrained.esm2_t33_650M_UR50D()
esm_650m_batch_converter = esm_650m_alphabet.get_batch_converter()
esm_650m_model.eval()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
esm_650m_model.to(device)
ic(device)
ic(torch.cuda.is_available())

ic| device: device(type='cuda', index=0)
ic| torch.cuda.is_available(): True


True

In [ ]:
class ESMProteinDataset(Dataset):
    def __init__(self,
                batch_converter,
                device,
                seqs_df):
        self.batch_converter = batch_converter
        self.device = device
        self.df = seqs_df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        uniprot_id = row['uniprot_id']
        wt_seq = row['wt_seq']
        _, _, batch_tokens = self.batch_converter([(uniprot_id, wt_seq)])
        return uniprot_id, batch_tokens[0].to(device)

torch.manual_seed(0)
esm_650m_dataset = ESMProteinDataset(esm_650m_batch_converter, device, seqs_df)
esm_650m_loader = DataLoader(esm_650m_dataset, batch_size=1, shuffle=True)

In [ ]:
for protein_ids, batch_tokens in tqdm(esm_650m_loader):
  with torch.no_grad():
    try:
      results = esm_650m_model(batch_tokens, repr_layers=[33])
    except RuntimeError as e:
      ic(e)
      ic(protein_ids)
      ic(protein_ids)
      continue
  token_embeddings = results["representations"][33]
  for i, protein_id in enumerate(protein_ids):
    index = uniprot_id_to_index[protein_id]
    row = seqs_df.iloc[index]
    esm_650m_embedding_path = row['esm_650m_embedding_path']
    averaged_embedding = token_embeddings[i].mean(dim=0)
    with open(esm_650m_embedding_path, 'wb') as f:
      torch.save(averaged_embedding, f)

100%|██████████| 1686/1686 [01:16<00:00, 21.91it/s]


## Create ESM 3B Embeddings

In [ ]:
esm_650m_model.cpu()
gc.collect()
torch.cuda.empty_cache()

esm_3b_model, esm_3b_alphabet = esm.pretrained.esm2_t36_3B_UR50D()
esm_3b_batch_converter = esm_3b_alphabet.get_batch_converter()
esm_3b_model.eval()
esm_3b_model.to(device)
ic(device)

ic| device: device(type='cuda', index=0)


device(type='cuda', index=0)

In [ ]:
torch.manual_seed(0)
esm_3b_dataset = ESMProteinDataset(esm_3b_batch_converter, device, seqs_df)
esm_3b_loader = DataLoader(esm_3b_dataset, batch_size=1, shuffle=True)

In [ ]:
for protein_ids, batch_tokens in tqdm(esm_3b_loader):
  with torch.no_grad():
    try:
      results = esm_3b_model(batch_tokens, repr_layers=[33])
    except RuntimeError as e:
      ic(e)
      ic(protein_ids)
      ic(protein_ids)
      continue
  token_embeddings = results["representations"][33]
  for i, protein_id in enumerate(protein_ids):
    index = uniprot_id_to_index[protein_id]
    row = seqs_df.iloc[index]
    esm_3b_embedding_path = row['esm_3b_embedding_path']
    averaged_embedding = token_embeddings[i].mean(dim=0)
    with open(esm_3b_embedding_path, 'wb') as f:
      torch.save(averaged_embedding, f)

100%|██████████| 1686/1686 [02:20<00:00, 11.99it/s]


## Create PTMMamba Embeddings

In [ ]:
class PTMProteinDataset(Dataset):
    def __init__(self,
                device,
                seqs_df):
        self.device = device
        self.df = seqs_df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        uniprot_id = row['uniprot_id']
        ptm_seq = row['ptm_seq']
        return uniprot_id, ptm_seq

torch.manual_seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
ptm_mamba_dataset = PTMProteinDataset(device, seqs_df)
ptm_mamba_loader = DataLoader(ptm_mamba_dataset, batch_size=1, shuffle=False)

In [ ]:
model_checkpoint_path = '/workspace/ckpt/bi_mamba-esm-ptm_token_input/best.ckpt'
mamba = PTMMamba(ckpt_path=model_checkpoint_path, device=device)

<All keys matched successfully>


In [ ]:
ptm_tokenizer = PTMTokenizer()

for protein_id, ptm_seq in tqdm(ptm_mamba_loader):
    ptm_seq = ptm_seq[0]
    protein_id = protein_id[0]
    index = uniprot_id_to_index[protein_id]
    ptm_mamba_embedding_path = seqs_df.iloc[index]['ptm_mamba_embedding_path']
    tokenized_output = ptm_tokenizer(ptm_seq, return_tensor=True)
    with torch.no_grad():
        try:
            output = mamba(ptm_seq)
            embedding = output.hidden_states.squeeze(dim=0)
            averaged_embedding = embedding.mean(dim=0)
            with open(ptm_mamba_embedding_path, 'wb') as f:
                torch.save(averaged_embedding, f)
        except RuntimeError as e:
            ic(e)
            ic(protein_id)

100%|██████████| 1686/1686 [02:14<00:00, 12.58it/s]
